# Reservoir operation optimisation
In this Notebook we will see how simulation models and optimisation algorithms can be used to assist the operation of a water reservoir system.

<left><img src="../../util/images/Dam3.gif" width = "700px"><left>
    
We consider a simple illustrative system where a reservoir is operated to supply water to a domestic consumption node, while ensuring a minimum environmental flow in the downstream river (also called “environmental compensation flow”) and maintaining the water level in the reservoir within prescribed limits. We use a mathematical model to link all the key variables that represent the reservoir dynamics (inflow, storage and outflows) and use model simulation/optimisation to determine the reservoir release scheduling that will best meet the water demand over a coming period of time, given the predicted (or assumed) scenario of future inflows.
<left> <img src="../../util/images/system_representation_IO0.png" width = "600px"><left>

## Import libraries
Before getting started, let's import some libraries that will be used throughout the Notebook:

In [1]:
from bqplot import pyplot as plt
from bqplot import *
from bqplot.traits import *
import numpy as np
import ipywidgets as widgets
from Modules.Interactive_release_schedule import Interactive_release_single,Interactive_release_double, Interactive_Pareto_front
warnings.filterwarnings('ignore') # to ignore warning messages

## The reservoir model

The mathematical model of the reservoir essentially consists of a water balance equation, where the storage (***s***) at a future time step (for example, at the beginning of the next week) is predicted from the storage at the current time (the beginning of the this week) by adding and subtracting the inflows and outflows that will occur during the temporal interval ahead:

$s(t+1) = s(t) + I(t) – E(t) – env(t) - spill(t) – Qreg(t)$   

<left> <img src="../../util/images/system_representation_IO1.png" width = "600px"><left>

#### Implementation of the reservoir simulation function
Here we define a function that implements the reservoir simulation, that is, iteratively apply the mass balance equation and reconstruct the temporal evolution of the reservoir variables over the simulation period

In [2]:
def syst_sim(N,I,e,d,S0,Smax,env_min,Qreg):

    # Declare output variables

    S = [0]*(N+1) # reservoir storage in ML

    spill = [0]*N # spillage in ML

    env = [env_min]*N # environmental compensation flow
    
    S[0] = S0 # initial storage

    for t in range(N): # Loop for each time-step (week)

        # If at week t the inflow (I) is lower than the minimum environmental compensation (env_min), 
        # then the environmental compensation (env) = inflow (I)  
        if env_min >= I[t] :
            env[t] = I[t]
        # If the minimum environmental compensation is higher than the water resource available (S + I - E)
        # then the environmental compensation is equal to the higher value between 0 and the resource available
        if env_min >= S[t] + I[t] - e[t]:
            env[t] = max(0,S[t] + I[t] - e[t]) 
        # If the demand is higher than the water resource available (S + I - E - env)
        # then the release for water supply is equal to the higher value between 0 and the resource available            
        if d[t] >= S[t] + I[t] - e[t] - env[t]:
            Qreg[t] = min(Qreg[t],max(0,S[t] + I[t] - e[t] - env[t]))
        # The spillage is equal to the higher value between 0 and the resource available exceeding the reservoir capacity
        spill[t] = max(0,S[t] + I[t] - Qreg[t] - env[t] - e[t] - Smax)
        # The final storage (initial storage in the next step) is equal to the storage + inflow - outflows
        S[t+1] = S[t] + I[t] - Qreg[t] - env[t]- e[t] - spill[t]
        
    return S,env,spill,Qreg

#### Definition of inflow and demand scenarios
Let's assume we want to look at the next 8 weeks the number of weeks (so ***N=8***), and assume we have forecasts of inflows and demand for this period.

In [3]:
N = 8 # (weeks) length of the simulation period

I_fore = np.array([15,17,19,11,9,4,3,8]) # (ML/week) time series of inflow forecasts
T_fore = np.array([13,13,17,18,20,22,25,26]) # (degC) time series of temperature forecasts
d_fore = np.array([15]*N)*T_fore/15 # (ML/week) time series of demand forecasts, estimated as a function of temperature
# (this is onviously a very simplified approach, and one could use a much more sophisticated demand model)

Plot the inflow and demand forecasts:

In [4]:
# Axis characterisitcs
x_sc_1 = LinearScale();y_sc_1 = LinearScale(min=0,max=35)
x_ax_1 = Axis(label='week', scale=x_sc_1);y_ax_1 = Axis(label='ML/week', scale=y_sc_1, orientation='vertical')
# Bar plot
inflow_plot = plt.bar(np.arange(1,N+1),I_fore,colors=['blue'],stroke = 'lightgray',scales={'x': x_sc_1, 'y': y_sc_1},
                      labels = ['inflow'], display_legend = True)
#Figure characteristics
fig_1a = plt.Figure(marks = [inflow_plot],title = 'Inflow forecast for the next 8 weeks', axes=[x_ax_1, y_ax_1],
                    layout={'min_width': '1000px', 'max_height': '300px'}, legend_style = {'fill': 'white', 'opacity': 0.5})
widgets.VBox([fig_1a])

In [5]:
# Bar plot (we use the same axis as the weekly inflows figure)
demand_plot   = plt.bar(np.arange(1,N+1),d_fore,colors=['gray'],stroke = 'lightgray',opacities = [1]*N, 
                        labels = ['demand'], display_legend = True, 
                    stroke_width = 1,scales={'x': x_sc_1, 'y': y_sc_1})
#Figure characteristics
fig_1b = plt.Figure(marks = [demand_plot],title = 'Demand forecast for the next 8 weeks', axes=[x_ax_1, y_ax_1],
                    layout={'min_width': '1000px', 'max_height': '300px'}, legend_style = {'fill': 'white', 'opacity': 1})
widgets.VBox([fig_1b])

#### Definition of other input parameters
Let's define other variables that are needed for the reservoir system simulation, such as the reservoir storage capacity, the environmental compensation flow, etc.

In [6]:
### Constraints ###
s_max = 150 #  (ML) Maximum storage (=reservoir capacity)
s_min = 0 # (ML) Minimum storage (set to zero for now)
env_min = 2 # (ML/week)   # Environmental compensation flow

### Initial conditions ###
s_0 = 80 # (ML) # Storage volume at the beginning of the simulation period
e_fore = T_fore*0.1 # (ML/week) Time series of evaporation from the reservoir (this is a very simplified 
# approach and could be replaced by a more realistic estimation approach)

## From single to multi-objective optimization
Now let's assume that, besides minimising supply deficits, we are also interested in minimising the chances that the reservoir level go below a minimum threshold. This could be, for example, because the quality of the water deteriorates when levels are low, requiring more costly treatment. We measure how well this criterion is satisfied by the following objective function:

$$MSV = \sum_{t=1}^{N} max ( \ rc - S(t) , \ 0)$$

where, again, N is the length of the simulation period, S is the reservoir storage, and rc is the minimum reservoir storage threshold that should preferably not be transpassed (MSV stands for Minimum Storage Violation). 

For our case, let's set this threshold to 30 ML:

In [7]:
# Minimum storage threshold
ms = np.array([30]*(N+1)) # in ML

## From manual to automatic optimization approach
As we have seen, when we deal with two conflicting objectives, we cannot find a solution that optimise both simoultaneously. If we prioritize one objective, the other one is deteriorated: there is a trade-off between the two. It would then be interesting to explore this tradeoff, and find all the release schedules that produce a different optimal combination of the two objectives. However, this is too cumbersome to do manually. Here we then use a multi-objective optimisation algorithm to do that for us. 

<left><img src="../../util/images/AutomaticOptimization2.gif" width = "500px"><left>
To this end, we use the Python Platypus package, and the NSGAII algorithm implemented in it. For more information about these methods and tools, see [Deb et al, 2002](https://ieeexplore.ieee.org/document/996017) and the [Platypus webpage](https://platypus.readthedocs.io). The code to run the optimisation is the following:

In [8]:
# Optimizer
from platypus import NSGAII, Problem, Real, Integer

def auto_optim(vars):
    release1 = vars[0]
    release2 = vars[1]
    release3 = vars[2]
    release4 = vars[3]
    release5 = vars[4]
    release6 = vars[5]
    release7 = vars[6]
    release8 = vars[7]

    Qreg = [release1,release2,release3,release4,release5,release6,release7,release8]
    s,env,spill,Qreg = syst_sim(N,I_fore,e_fore,d_fore,s_0,s_max,env_min,Qreg)
    
    sdpen = (np.sum((np.maximum(d_fore-Qreg,[0]*N))**2)).astype('int')
    lspen = (np.sum((np.maximum(ms-s,[0]*(N+1))))).astype('int')
    
    return [sdpen,lspen]

problem = Problem(N,2)
Real0 = Real(0, 40);Real1 = Real(0, 40);Real2 = Real(0, 40);Real3 = Real(0, 40);
Real4 = Real(0, 40);Real5 = Real(0, 40);Real6 = Real(0, 40);Real7 = Real(0, 40)

problem.types[:] = [Real0] + [Real1] + [Real2] + [Real3] + [Real4] + [Real5] + [Real6] + [Real7] 
problem.function = auto_optim

population_size = 20
algorithm = NSGAII(problem,population_size)
algorithm.run(10000) # Number of iterations

results1_optim_relea = np.array([algorithm.result[i].objectives[0] for i in range(population_size)])
results2_optim_relea = np.array([algorithm.result[i].objectives[1] for i in range(population_size)])

solutions_optim_relea = [algorithm.result[i].variables[0:N] for i in range(population_size)]

#### Plot the optimisation results
We can visualise the tradeoffs between the two objectives in one plot, called Pareto front, which displays the combination of the two objective values in correspondence to a set of optimised solutions. Click on one point in the Pareto front to visualise the release scheduling that generates that performance, and associated storage time series.  What do you think would be a balanced solution?

In [9]:
# Interactive Pareto front
fig_4a,fig_4b,fig_pf = Interactive_Pareto_front(
    N,I_fore,e_fore,d_fore,s_0,s_max,ms,env_min, demand_plot,solutions_optim_relea,results1_optim_relea,results2_optim_relea)
widgets.VBox([widgets.HBox([widgets.VBox([fig_4b,fig_4a]),fig_pf])])

### References 

Deb K. et al (2002) A fast and elitist multiobjective genetic algorithm: NSGA-II, IEEE Transactions on Evolutionary Computation, 6(2), 182-197, doi:10.1109/4235.996017.

You J.Y. and Cai X. (2008) Hedging rule for reservoir operations: 1. A theoretical analysis, Water Resour. Res., 44, W01415, doi:10.1029/2006WR005481.

#### Let's go to the next section!: [5. Decision making under uncertainty](5.%20Decision%20making%20under%20uncertainty.ipynb)